In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1m4kXsnHUKwyMYk1HQckaT2h57_geHkyq"})  
downloaded.GetContentFile('attention.py')

In [0]:
import attention
from attention import AttentionLayer

In [8]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:

downloaded = drive.CreateFile({'id':"1_81cqoIIq0EdZpm7vFR962SuIzODMkpN"})  
downloaded.GetContentFile('Reviews.csv')

In [0]:
import pandas as pd
data=pd.read_csv("Reviews.csv",nrows=100000)

In [0]:
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [13]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def clean_text(text,num):
    ns = text.lower()
    ns = BeautifulSoup(ns, "lxml").text
    ns = re.sub(r'\([^)]*\)', '', ns)
    ns = re.sub('"','', ns)
    ns = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in ns.split(" ")])    
    ns = re.sub(r"'s\b","",ns)
    ns = re.sub("[^a-zA-Z]", " ", ns) 
    ns = re.sub('[m]{2,}', 'mm', ns)
    if(num==0):
        tokens = [w for w in ns.split() if not w in stop_words]
    else:
        tokens=ns.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [15]:
clean_txt = []
for t in data['Text']:
    clean_txt.append(clean_text(t,0))

clean_summary = []
for t in data['Summary']:
    clean_summary.append(clean_text(t,1))

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.amazon.com/gp/product/b007i7yygy/ref=cm_cr_rev_prod_title" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [0]:
data['clean_txt']=clean_txt
data['clean_summary']=clean_summary

In [0]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
max_text=30
max_summ=8

In [0]:
clean_txt =np.array(data['clean_txt'])
clean_summary=np.array(data['clean_summary'])

In [0]:
short_txt=[]
short_summ=[]

for i in range(len(clean_txt)):
    if(len(clean_summary[i].split())<=max_summ and len(clean_txt[i].split())<=max_text):
        short_txt.append(clean_txt[i])
        short_summ.append(clean_summary[i])
        
df=pd.DataFrame({'text':short_txt,'summary':short_summ})

In [0]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True)

In [24]:
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_train))
thresh=4
count=0
tot_count=0
freq=0
tot_freq=0
for key,value in x_tokenizer.word_counts.items():
    tot_count=tot_count+1
    tot_freq=tot_freq+value
    if(value<thresh):
        count=count+1
        freq=freq+value
print("% of rare words in vocabulary:",(count/tot_count)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 66.12339930151339
Total Coverage of rare words: 2.953684513790566


In [25]:
x_tokenizer = Tokenizer(num_words=tot_count-count) 
x_tokenizer.fit_on_texts(list(x_train))
x_train_seq    =   x_tokenizer.texts_to_sequences(x_train) 
x_test_seq   =   x_tokenizer.texts_to_sequences(x_test)
x_train    =   pad_sequences(x_train_seq,  maxlen=max_text, padding='post')
x_test   =   pad_sequences(x_test_seq, maxlen=max_text, padding='post')
x_voc   =  x_tokenizer.num_words + 1

x_voc

8440

In [26]:
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_train))

thresh=6
count=0
tot_count=0
freq=0
tot_freq=0
for key,value in y_tokenizer.word_counts.items():
    tot_count=tot_count+1
    tot_freq=tot_freq+value
    if(value<thresh):
        count=count+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(count/tot_count)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 78.12740675541863
Total Coverage of rare words: 5.3921899389571895


In [27]:
y_tokenizer = Tokenizer(num_words=tot_count-count) 
y_tokenizer.fit_on_texts(list(y_train))
y_train_seq    =   y_tokenizer.texts_to_sequences(y_train) 
y_test_seq   =   y_tokenizer.texts_to_sequences(y_test) 
y_train    =   pad_sequences(y_train_seq, maxlen=max_summ, padding='post')
y_test   =   pad_sequences(y_test_seq, maxlen=max_summ, padding='post')
y_voc  =   y_tokenizer.num_words +1
y_voc

1989

In [28]:
y_tokenizer.word_counts['sostok'],len(y_train)

(42453, 42453)

In [0]:
ind=[]
for i in range(len(y_train)):
    count=0
    for j in y_train[i]:
        if j!=0:
            count=count+1
    if(count==2):
        ind.append(i)
y_train=np.delete(y_train,ind, axis=0)
x_train=np.delete(x_train,ind, axis=0)

In [0]:
ind=[]
for i in range(len(y_test)):
    count=0
    for j in y_test[i]:
        if j!=0:
            count=count+1
    if(count==2):
        ind.append(i)
y_test=np.delete(y_test,ind, axis=0)
x_test=np.delete(x_test,ind, axis=0)

In [31]:
from keras import backend as K 
K.clear_session()
latent_dim = 300
embedding_dim=100

encoder_inputs = Input(shape=(max_text,))
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#lstm 1
enc_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = enc_lstm1(enc_emb)

#lstm 2
enc_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = enc_lstm2(encoder_output1)

#lstm 3
enc_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= enc_lstm3(encoder_output2)

decoder_inputs = Input(shape=(None,))

dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      844000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 30, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [0]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [34]:
x_train.shape

(41346, 30)

In [35]:
history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=128, validation_data=([x_test,y_test[:,:-1]], y_test.reshape(y_test.shape[0],y_test.shape[1], 1)[:,1:]))

Epoch 1/50
324/324 [==============================] - 206s 637ms/step - loss: 2.8156 - val_loss: 2.5738
Epoch 2/50
324/324 [==============================] - 206s 635ms/step - loss: 2.4905 - val_loss: 2.4418
Epoch 3/50
324/324 [==============================] - 205s 634ms/step - loss: 2.3363 - val_loss: 2.3690
Epoch 4/50
324/324 [==============================] - 205s 633ms/step - loss: 2.2350 - val_loss: 2.2590
Epoch 5/50
324/324 [==============================] - 204s 629ms/step - loss: 2.1655 - val_loss: 2.1939
Epoch 6/50
324/324 [==============================] - 206s 636ms/step - loss: 2.1057 - val_loss: 2.1836
Epoch 7/50
324/324 [==============================] - 208s 641ms/step - loss: 2.0554 - val_loss: 2.1356
Epoch 8/50
324/324 [==============================] - 206s 637ms/step - loss: 2.0094 - val_loss: 2.1346
Epoch 9/50
324/324 [==============================] - 208s 643ms/step - loss: 1.9675 - val_loss: 2.0908
Epoch 10/50
324/324 [==============================] - 208s 641m

In [0]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [0]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

In [0]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text,latent_dim))

In [0]:
dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

decoder_outputs2 = decoder_dense(decoder_inf_concat) 

In [0]:
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [0]:
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_word_index['sostok']

    stop = False
    decoded_sentence = ''
    while not stop:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summ-1)):
            stop = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    ns=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            ns=ns+reverse_target_word_index[i]+' '
    return ns

def seq2text(input_seq):
    ns=''
    for i in input_seq:
        if(i!=0):
            ns=ns+reverse_source_word_index[i]+' '
    return ns

In [43]:
for i in range(0,100):
    print("Review:",seq2text(x_train[i]))
    print("Original summary:",seq2summary(y_train[i]))
    print("Predicted summary:",decode_sequence(x_train[i].reshape(1,max_text)))
    print("\n")


Review: gave caffeine shakes heart anxiety attack plus tastes unbelievably bad stick coffee tea soda thanks 
Original summary: hour 
Predicted summary:  good stuff


Review: got great course good belgian chocolates better 
Original summary: would like to give it stars but 
Predicted summary:  yummy


Review: one best flavored coffees tried usually like flavored coffees one great serve company love 
Original summary: delicious 
Predicted summary:  great coffee


Review: salt separate area pain makes hard regulate salt putting like salt go ahead get product 
Original summary: tastes ok packaging 
Predicted summary:  salt


Review: really like product super easy order online delivered much cheaper buying gas station stocking good long drives 
Original summary: turkey jerky is great 
Predicted summary:  great product


Review: best salad dressing delivered promptly quantities last vidalia onion dressing compares made oak hill farms sometimes find costco order front door want even orders cu